# 1. Imports

In [58]:
import psycopg2 as pg
import pandas   as pd
import os 

from dotenv import load_dotenv

In [62]:
# DB Credentials
load_dotenv()

host = os.environ.get( 'host' )
port = os.environ.get( 'port' )
database = os.environ.get( 'database' )
username = os.environ.get( 'user' )
pwd = os.environ.get( 'pwd' )

## 1.1. Collect Data Using Psycopg2

In [24]:
# create DB connection
conn = pg.connect( user=username,
                   password=pwd,
                   host=host,
                   port=port,
                   database=database )

### 1.1.1. Schema Query

In [21]:
# create cursor and execute query
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute( query_schema )
record = cursor.fetchall()
record

[('pg_toast',),
 ('pg_temp_1',),
 ('pg_toast_temp_1',),
 ('pg_catalog',),
 ('information_schema',),
 ('public',),
 ('pa004',),
 ('pg_temp_4',),
 ('pg_toast_temp_4',),
 ('pa005',),
 ('pg_temp_9',),
 ('pg_toast_temp_9',)]

### 1.1.2. Tables Query 

In [22]:
# create cursor and execute query
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='pa004'
"""

cursor.execute( query_tables )
record = cursor.fetchall()
record

[('users',), ('vehicle',), ('insurance',)]

### 1.1.3. Collect Data

In [41]:
# create cursor and execute query
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    LIMIT 10
"""

cursor.execute( query_tables_users )
record = cursor.fetchall()
record

[(1, 'Male', 44, 28.0, 26.0),
 (2, 'Male', 76, 3.0, 26.0),
 (3, 'Male', 47, 28.0, 26.0),
 (4, 'Male', 21, 11.0, 152.0),
 (5, 'Female', 29, 41.0, 152.0),
 (6, 'Female', 24, 33.0, 160.0),
 (7, 'Male', 23, 11.0, 152.0),
 (8, 'Female', 56, 28.0, 26.0),
 (9, 'Female', 24, 3.0, 152.0),
 (10, 'Female', 32, 6.0, 152.0)]

In [42]:
# close cursor and connection
cursor.close()
conn.close()

In [43]:
# convert to dataframe
data = pd.DataFrame( record )
data.head()

,0,1,2,3,4
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


## 1.2. Collect Data Using Pandas

In [63]:
# create DB connection
conn = pg.connect( user=username,
                   password=pwd,
                   host=host,
                   port=port,
                   database=database )

In [64]:
# users table query
query = """
    SELECT *
    FROM pa004.users
    LIMIT 10
"""

df = pd.read_sql( query, conn )
df.head()

,id,gender,age,region_code,policy_sales_channel
0,1,Male,44,28.0,26.0
1,2,Male,76,3.0,26.0
2,3,Male,47,28.0,26.0
3,4,Male,21,11.0,152.0
4,5,Female,29,41.0,152.0


In [49]:
# vehicle table query
query = """
    SELECT *
    FROM pa004.vehicle
    LIMIT 10
"""

df = pd.read_sql( query, conn )
df.head()

,id,driving_license,vehicle_age,vehicle_damage
0,1,1,> 2 Years,Yes
1,2,1,1-2 Year,No
2,3,1,> 2 Years,Yes
3,4,1,< 1 Year,No
4,5,1,< 1 Year,No


In [50]:
# insurance table query
query = """
    SELECT *
    FROM pa004.insurance
    LIMIT 10
"""

df = pd.read_sql( query, conn )
df.head()

,id,previously_insured,annual_premium,vintage,response
0,1,0,40454.0,217,1
1,2,0,33536.0,183,0
2,3,0,38294.0,27,1
3,4,1,28619.0,203,0
4,5,1,27496.0,39,0


In [55]:
# join tables
query = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON ( u.id = v.id )
                       LEFT JOIN pa004.insurance i ON ( u.id = i.id )
"""

df_raw = pd.read_sql( query, conn )
df_raw.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.0,152.0,7,1,< 1 Year,Yes,7,0,23367.0,249,0
1,13,Female,41,15.0,14.0,13,1,1-2 Year,No,13,1,31409.0,221,0
2,18,Female,25,35.0,152.0,18,1,< 1 Year,No,18,1,46622.0,299,0
3,31,Female,26,8.0,160.0,31,1,< 1 Year,No,31,0,2630.0,136,0
4,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0


In [52]:
# close cursor and connection
cursor.close()
conn.close()